# IRIS Data Analysis - Structural Schema & Relationship Mapping

This notebook focuses on identifying the database schema, Primary Keys (PK), and Foreign Keys (FK) to determine table relationships and isolate irrelevant (junk) files.

In [1]:
import pandas as pd
import glob
import os
import itertools

pd.set_option('display.max_columns', None)

## 1. Load and Clean Data
Loading all CSVs and dropping the `index` column immediately.

In [2]:
DATA_PATH = 'Sales Dataset'
csv_files = glob.glob(os.path.join(DATA_PATH, "*.csv"))
dataframes = {}

for file in csv_files:
    filename = os.path.basename(file)
    try:
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='ISO-8859-1')
            
        # Drop 'index' column if it exists
        if 'index' in df.columns:
            df = df.drop(columns=['index'])
            
        # Standardize column names (strip whitespace, lower case for comparison)
        # But keep original for display
        df.columns = [c.strip() for c in df.columns]
            
        dataframes[filename] = df
        print(f"Loaded {filename}: {df.shape}")
    except Exception as e:
        print(f"Error loading {filename}: {e}")

C:\Users\ashis\AppData\Local\Temp\ipykernel_11836\773723843.py:9: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8')


Loaded Amazon Sale Report.csv: (128975, 23)
Loaded Cloud Warehouse Compersion Chart.csv: (50, 3)
Loaded Expense IIGF.csv: (17, 4)
Loaded International sale Report.csv: (37432, 9)
Loaded May-2022.csv: (1330, 16)
Loaded P  L March 2021.csv: (1330, 17)
Loaded Sale Report.csv: (9271, 6)


## 2. Identify Primary Keys (PK)
A Primary Key must be unique and non-null. We will check each column in every dataframe.

In [3]:
potential_pks = {}

print("--- Potential Primary Keys ---")
for name, df in dataframes.items():
    pks = []
    for col in df.columns:
        if df[col].is_unique and not df[col].isnull().any():
            pks.append(col)
    potential_pks[name] = pks
    print(f"{name}: {pks}")

--- Potential Primary Keys ---


Amazon Sale Report.csv: []
Cloud Warehouse Compersion Chart.csv: []
Expense IIGF.csv: ['Unnamed: 3']
International sale Report.csv: []
May-2022.csv: ['Sku']
P  L March 2021.csv: ['Sku']
Sale Report.csv: []


## 3. Identify Relationships (Foreign Keys)
We look for columns that share names and data content between tables.

In [4]:
print("--- Relationship Mapping (Shared Columns) ---")
file_pairs = itertools.combinations(dataframes.keys(), 2)
connections = []

for name1, name2 in file_pairs:
    cols1 = set(dataframes[name1].columns)
    cols2 = set(dataframes[name2].columns)
    
    # Find common columns
    common_cols = cols1.intersection(cols2)
    
    # Also check for fuzzy matches (e.g. 'SKU' vs 'Sku' vs 'SKU Code')
    # We'll normalize to lower case for this check
    cols1_lower = {c.lower(): c for c in cols1}
    cols2_lower = {c.lower(): c for c in cols2}
    common_lower = set(cols1_lower.keys()).intersection(set(cols2_lower.keys()))
    
    for c_lower in common_lower:
        c1 = cols1_lower[c_lower]
        c2 = cols2_lower[c_lower]
        
        # Verify content overlap to confirm it's a real relationship
        vals1 = set(dataframes[name1][c1].dropna().unique())
        vals2 = set(dataframes[name2][c2].dropna().unique())
        
        overlap = vals1.intersection(vals2)
        if len(overlap) > 0:
            print(f"{name1} ({c1}) <-> {name2} ({c2}) | Overlap: {len(overlap)}")
            connections.append((name1, name2))
        else:
             print(f"{name1} ({c1}) <-> {name2} ({c2}) | No content overlap (False Positive)")

--- Relationship Mapping (Shared Columns) ---
Amazon Sale Report.csv (Size) <-> International sale Report.csv (Size) | Overlap: 10
Amazon Sale Report.csv (SKU) <-> International sale Report.csv (SKU) | Overlap: 3699
Amazon Sale Report.csv (Style) <-> International sale Report.csv (Style) | Overlap: 977


Amazon Sale Report.csv (Date) <-> International sale Report.csv (DATE) | Overlap: 25
Amazon Sale Report.csv (SKU) <-> May-2022.csv (Sku) | No content overlap (False Positive)
Amazon Sale Report.csv (Category) <-> May-2022.csv (Category) | No content overlap (False Positive)
Amazon Sale Report.csv (SKU) <-> P  L March 2021.csv (Sku) | No content overlap (False Positive)
Amazon Sale Report.csv (Category) <-> P  L March 2021.csv (Category) | No content overlap (False Positive)
Amazon Sale Report.csv (Category) <-> Sale Report.csv (Category) | No content overlap (False Positive)
Amazon Sale Report.csv (Size) <-> Sale Report.csv (Size) | Overlap: 9
Cloud Warehouse Compersion Chart.csv (Unnamed: 1) <-> Expense IIGF.csv (Unnamed: 1) | No content overlap (False Positive)
International sale Report.csv (SKU) <-> May-2022.csv (Sku) | No content overlap (False Positive)
International sale Report.csv (SKU) <-> P  L March 2021.csv (Sku) | No content overlap (False Positive)
International sale Report

## 4. Isolate Junk Tables
Tables that have NO connections to others are likely junk or standalone reference files.

In [5]:
connected_files = set()
for n1, n2 in connections:
    connected_files.add(n1)
    connected_files.add(n2)
    
all_files = set(dataframes.keys())
junk_files = all_files - connected_files

print("\n--- Connected Files (Core Schema) ---")
for f in connected_files:
    print(f)

print("\n--- Isolated Files (Potential Junk) ---")
for f in junk_files:
    print(f)
    print(f"Preview of {f}:")
    display(dataframes[f].head())


--- Connected Files (Core Schema) ---
P  L March 2021.csv
Sale Report.csv
International sale Report.csv
May-2022.csv
Amazon Sale Report.csv

--- Isolated Files (Potential Junk) ---
Expense IIGF.csv
Preview of Expense IIGF.csv:


,Recived Amount,Unnamed: 1,Expance,Unnamed: 3
0,Particular,Amount,Particular,Amount
1,06-19-22,1000,Large Bag,380
2,06-20-22,1500,"Stationary(Soft Pin, Paper pin for Dupatta, Fe...",170
3,06-22-22,500,OLA,839
4,06-23-22,2000,Auto Rent,520


Cloud Warehouse Compersion Chart.csv
Preview of Cloud Warehouse Compersion Chart.csv:


,Shiprocket,Unnamed: 1,INCREFF
0,Heads,Price (Per Unit),Price (Per Unit)
1,Inbound (Fresh Stock and RTO),₹4.00,4
2,Outbound,₹7.00,11
3,Storage Fee/Cft,₹25.00,Rs 0.15/- Per Day
4,Customer Return with Detailed QC,₹6.00,15.5
